### 一，Autograph编码规范总结


* 1，被@tf.function修饰的函数应尽可能使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.

* 2，避免在@tf.function修饰的函数内部定义tf.Variable. 

* 3，被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。

### 二，Autograph编码规范解析


 **1，被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数。**

In [1]:
import numpy as np 
import tensorflow as tf 

@tf.function
def np_random():
    a = np.random.randn(3, 3)
    tf.print(a)

@tf.function
def tf_random():
    a = tf.random.normal([3, 3])
    tf.print(a)

In [2]:
#np_random每次执行都是一样的结果。
np_random()
np_random()

array([[ 0.77449502,  3.00146112,  1.02022741],
       [-0.75623601, -2.13654338, -1.21120147],
       [ 0.04550913, -0.32167834, -1.08226863]])
array([[ 0.77449502,  3.00146112,  1.02022741],
       [-0.75623601, -2.13654338, -1.21120147],
       [ 0.04550913, -0.32167834, -1.08226863]])


In [3]:
#tf_random每次执行都会有重新生成随机数。
tf_random()
tf_random()

[[1.33254421 -0.790196121 1.80904615]
 [0.598678887 1.56947219 0.267028302]
 [0.41277492 -0.0901161805 0.466290832]]
[[-0.0303624421 0.268078417 0.607211351]
 [1.44866574 0.997466862 -0.0985851362]
 [0.539642036 0.0878665447 -0.299904674]]


**2，避免在@tf.function修饰的函数内部定义tf.Variable.**

In [4]:
# 避免在@tf.function修饰的函数内部定义tf.Variable.

x = tf.Variable(1.0,dtype=tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var() 
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [7]:
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

#执行将报错
#inner_var()
#inner_var()



**3,被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量。**

In [10]:
tensor_list = []

#@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [11]:
tensor_list = []

@tf.function #加上这一行切换成Autograph结果将不符合预期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list


append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
